In [9]:
import os
from jwst.pipeline import Detector1Pipeline, Image2Pipeline, Coron3Pipeline
import jwst.associations
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
from glob import glob
import astropy.io.fits as fits
import numpy as np
import time
import pandas as pd
import json
import pickle
from json import JSONEncoder
from collections import defaultdict
from CoronPipeline import MyCoron3Pipeline

In [10]:
os.environ["CRDS_PATH"] = '/home/sarperyn/crds_cache/jwst_ops'
os.environ["CRDS_SERVER_URL"] = 'https://jwst-crds.stsci.edu'

In [11]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

# figure sizes and resolutions
rcParams['figure.figsize'] = [20,15]
rcParams['figure.dpi'] = 80
rcParams['savefig.dpi'] = 80

In [12]:
import jwst
print(jwst.__version__)

1.8.3.dev39+g731bc17


In [13]:
PROPOSAL_ID = '1441'

# Pipeline things



<div class="alert alert-block alert-info">
Now we have the fits files. We will create our coronagrahic images using the JWST pipeline. Our data has the suffix <b>.rateints</b> just like we needed. 
While processing coronagraphy data, the common procedure is using the following pipeline:
    <ul> 
        <li> calwebb_detector1 $\rightarrow$ <strong>from jwst.pipeline import Detector1Pipeline</strong>
        <li> calwebb_image2  $\rightarrow$ <strong>from jwst.pipeline import Image2Pipeline</strong>
        <li> calwebb_coron3  $\rightarrow$ <strong>from jwst.pipeline import Coron3Pipeline</strong>
    </ul>
</div>

We have the outputs of the first stage. They all have the `rateints.fits` suffix. By passing them through the second pipeline we are going to end up with `calints.fits` files which is a last step before getting the coron products.

In [14]:
# Path containing the rateints data
directory = f'/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/{PROPOSAL_ID}/mastDownload/JWST/'

### Stage2

In [7]:
def runimg2(filename, output_dir):
    
    img2 = Image2Pipeline()
    img2.output_dir = output_dir
    img2.save_results = True
    img2(filename)

In [22]:
rateints_files = glob(os.path.join(directory, '**/*rateints.fits'))

In [9]:
batch_size = 5

In [8]:
#rateints_files

In [46]:
for i in range(0,len(rateints_files),batch_size):
    
    for f in rateints_files[i:i+batch_size]:
        
        output_dir = f[:117]
        runimg2(f,output_dir)
    time.sleep(1.5)

2022-12-04 07:05:42,657 - stpipe.Image2Pipeline - INFO - Image2Pipeline instance created.
2022-12-04 07:05:42,661 - stpipe.Image2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2022-12-04 07:05:42,664 - stpipe.Image2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2022-12-04 07:05:42,666 - stpipe.Image2Pipeline.flat_field - INFO - FlatFieldStep instance created.
2022-12-04 07:05:42,669 - stpipe.Image2Pipeline.photom - INFO - PhotomStep instance created.
2022-12-04 07:05:42,672 - stpipe.Image2Pipeline.resample - INFO - ResampleStep instance created.
2022-12-04 07:05:42,821 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline running with args ('/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/1386/mastDownload/JWST/jw01386001001_0310a_00002_nrcalong/jw01386001001_0310a_00002_nrcalong_rateints.fits',).
2022-12-04 07:05:42,826 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 

In [15]:
calints_data = glob(os.path.join(directory, '**/*_calints.fits'))

In [16]:
len(calints_data)

90

In [21]:
calints1 = fits.open(calints_data[0])
calints1.info()

Filename: /home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/1441/mastDownload/JWST/jw01441011001_03108_00001_nrcalong/jw01441011001_03108_00001_nrcalong_calints.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     356   ()      
  1  SCI           1 ImageHDU        97   (320, 320, 16)   float32   
  2  ERR           1 ImageHDU        11   (320, 320, 16)   float32   
  3  DQ            1 ImageHDU        12   (320, 320, 16)   int32 (rescales to uint32)   
  4  AREA          1 ImageHDU         9   (320, 320)   float32   
  5  INT_TIMES     1 BinTableHDU     24   16R x 7C   [J, D, D, D, D, D, D]   
  6  VAR_POISSON    1 ImageHDU        10   (320, 320, 16)   float32   
  7  VAR_RNOISE    1 ImageHDU        10   (320, 320, 16)   float32   
  8  VAR_FLAT      1 ImageHDU        10   (320, 320, 16)   float32   
  9  ASDF          1 BinTableHDU     11   1R x 1C   [26123B]   


### Stage 3

The stage 3 is kinda different than the other steps. To run this step we need an association (ASN) file. Thus, we need some functions to create ASN files.

In [25]:
from jwst.associations.mkpool import mkpool
from jwst.associations.lib.rules_level3 import Asn_Lv3Coron
from jwst.associations import AssociationPool, AssociationRegistry
from jwst.associations.mkpool import from_cmdline, mkpool
from jwst.associations import generate
from astropy.table import Table

In [26]:
pool = mkpool(calints_data)

In [27]:
pool_df = pool.to_pandas()
pool_df.to_csv(f'calints_{PROPOSAL_ID}_pool.csv',index=False)

In [28]:
def t_path(partial_path):
    __file__ = '/home/sarperyn/sarperyurtseven/ProjectFiles/notebooks/asn_coron_rule.py'
    test_dir = os.path.dirname(__file__)
    return os.path.join(test_dir, partial_path)

In [29]:
registry = AssociationRegistry([t_path('asn_coron_rule.py')],
                              include_default=False)

In [30]:
asns = generate(pool,registry)

202212051416:WARNING:jwst.associations.lib.rules_level3_base.rename_to_level2:Item FILENAME="/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/1441/mastDownload/JWST/jw01441011001_03108_00001_nrcalong/jw01441011001_03108_00001_nrcalong_calints.fits" is not a Level 1b name. Cannot transform to Level 2b.
202212051416:WARNING:jwst.associations.lib.rules_level3_base.rename_to_level2:Item FILENAME="/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/1441/mastDownload/JWST/jw01441011001_03108_00002_nrcalong/jw01441011001_03108_00002_nrcalong_calints.fits" is not a Level 1b name. Cannot transform to Level 2b.
202212051416:WARNING:jwst.associations.lib.rules_level3_base.rename_to_level2:Item FILENAME="/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/1441/mastDownload/JWST/jw01441011001_03108_00003_nrcalong/jw01441011001_03108_00003_nrcalong_calints.fits" is not a Level 1b name. Cannot transform to Level 2b.
202212051416:WARNING:jwst.associations.lib.rules_level3_base.

In [22]:
class FileEncoder(JSONEncoder):
        def default(self, o):
            return o.__dict__

In [23]:
def get_asns(asns):
    
    asn_list = []
    for idx,asn in enumerate(asns):
        asn_dict = {}
        
        for i,j in zip(asn.keys(),asn.values()):
            asn_dict[i] = j
            
        asn_list.append(asn_dict)
        
        
        # I couldn't figure out how to serialize the abc.AsnCoron_Lv3 object and save it as a json file
        # For a temporary solution we will have a list that contains asn files as dictionaries
        # In this way, we can easily run the pipeline
        '''
        FileEncoder().encode(asn_dict)
        with open(f'asnfile_{idx}.json','w') as output:
            encodedJSON = json.dump(asn_dict,output,indent=4,cls=FileEncoder)
        '''
        
    return asn_list
            

In [31]:
def runcoron(filename, output_dir):

    coron = MyCoron3Pipeline()
    coron.output_dir = output_dir
    coron.save_results = True
    coron.process(filename)

In [32]:
def get_stage3_products(asns,directory):
    
    for t in range(len(asns)):

        asn_dict = {}
        for i,j in zip(asns[t].keys(),asns[t].values()):
            asn_dict[i] = j
            
        runcoron(asn_dict,directory)
        
        

In [153]:
print(len(asns[0]['products'][0]['members']))
print(len(asns[1]['products'][0]['members']))
print(len(asns[2]['products'][0]['members']))

17

In [107]:
get_stage3_products(asns,directory)

2022-12-04 17:29:06,861 - stpipe.MyCoron3Pipeline - INFO - MyCoron3Pipeline instance created.
2022-12-04 17:29:06,864 - stpipe.MyCoron3Pipeline.stack_refs - INFO - StackRefsStep instance created.
2022-12-04 17:29:06,867 - stpipe.MyCoron3Pipeline.align_refs - INFO - AlignRefsStep instance created.
2022-12-04 17:29:06,870 - stpipe.MyCoron3Pipeline.klip - INFO - KlipStep instance created.
2022-12-04 17:29:06,873 - stpipe.MyCoron3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2022-12-04 17:29:06,876 - stpipe.MyCoron3Pipeline.resample - INFO - ResampleStep instance created.
2022-12-04 17:29:06,877 - stpipe.MyCoron3Pipeline - INFO - Starting calwebb_coron3 ...
2022-12-04 17:29:11,156 - stpipe.MyCoron3Pipeline - INFO - Prefetching reference files for dataset: 'jw01441007001_03106_00004_nrcalong_calints.fits' reftypes = ['drizpars', 'psfmask']
2022-12-04 17:29:11,160 - stpipe.MyCoron3Pipeline - INFO - Prefetch for DRIZPARS reference file is '/home/sarperyn/crds_cach

Output: jw01441-a3001_t004_nircam_f335m-maskbar-sub320alwb
Prod: {'name': 'jw01441-a3001_t004_nircam_f335m-maskbar-sub320alwb', 'members': [{'expname': '/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/1441/mastDownload/JWST/jw01441005001_03106_00001_nrcalong/jw01441005001_03106_00001_nrcalong_calints.fits', 'exptype': 'science', 'exposerr': None, 'asn_candidate': "[('o005', 'observation')]"}, {'expname': '/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/1441/mastDownload/JWST/jw01441006001_03106_00001_nrcalong/jw01441006001_03106_00001_nrcalong_calints.fits', 'exptype': 'science', 'exposerr': None, 'asn_candidate': "[('o006', 'observation')]"}, {'expname': '/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/1441/mastDownload/JWST/jw01441007001_03106_00004_nrcalong/jw01441007001_03106_00004_nrcalong_calints.fits', 'exptype': 'psf', 'exposerr': None, 'asn_candidate': "[('o007', 'observation')]"}, {'expname': '/home/sarperyn/sarperyurtseven/ProjectFiles/datase

2022-12-04 17:29:11,313 - stpipe.MyCoron3Pipeline - INFO - Prefetching reference files for dataset: 'jw01441007001_03106_00002_nrcalong_calints.fits' reftypes = ['drizpars', 'psfmask']
2022-12-04 17:29:11,317 - stpipe.MyCoron3Pipeline - INFO - Prefetch for DRIZPARS reference file is '/home/sarperyn/crds_cache/jwst_ops/references/jwst/nircam/jwst_nircam_drizpars_0001.fits'.
2022-12-04 17:29:11,318 - stpipe.MyCoron3Pipeline - INFO - Prefetch for PSFMASK reference file is '/home/sarperyn/crds_cache/jwst_ops/references/jwst/nircam/jwst_nircam_psfmask_0003.fits'.
2022-12-04 17:29:11,478 - stpipe.MyCoron3Pipeline - INFO - Prefetching reference files for dataset: 'jw01441007001_03106_00005_nrcalong_calints.fits' reftypes = ['drizpars', 'psfmask']
2022-12-04 17:29:11,482 - stpipe.MyCoron3Pipeline - INFO - Prefetch for DRIZPARS reference file is '/home/sarperyn/crds_cache/jwst_ops/references/jwst/nircam/jwst_nircam_drizpars_0001.fits'.
2022-12-04 17:29:11,482 - stpipe.MyCoron3Pipeline - INFO - 

KeyboardInterrupt: 